In [2]:
import tushare as ts
pro = ts.pro_api('c1dfcd419324efa15b3b2269de1180befae1562f928410e3900cfe04')

1、获取一日的不同类型交易者的主动买入占比

In [44]:
def get_daily_buy_per(trade_date):
    """
    获取一日的不同类型交易者的主动买入占比
    :param trade_date: 交易日期
    :return: 三种交易者的主动买入占比额
    """
    # 获取单日全部股票数据（单位：万元）
    data = pro.moneyflow(trade_date=trade_date)
    data.set_index('ts_code', drop=True, inplace=True)
    data['大单买入'] = data['buy_elg_amount']
    data['中单买入'] = data['buy_lg_amount']
    data['小单买入'] = data['buy_sm_amount'] + data['buy_md_amount']
    data = data.loc[:, ['大单买入', '中单买入', '小单买入']]
    sm_net_buy = data.sum()['大单买入'] / data.sum().sum()
    md_net_buy = data.sum()['中单买入'] / data.sum().sum()
    bg_net_buy = data.sum()['大单买入'] / data.sum().sum()

    return sm_net_buy, md_net_buy, bg_net_buy

2、获取开盘的日期

In [51]:
date_list_df = pro.trade_cal(exchange='SSE', start_date='20180101', end_date='20230401')
date_list_df = date_list_df[date_list_df['is_open'] == 1]
date_list = list(date_list_df['cal_date'])

['20230331',
 '20230330',
 '20230329',
 '20230328',
 '20230327',
 '20230324',
 '20230323',
 '20230322',
 '20230321',
 '20230320',
 '20230317',
 '20230316',
 '20230315',
 '20230314',
 '20230313',
 '20230310',
 '20230309',
 '20230308',
 '20230307',
 '20230306',
 '20230303',
 '20230302',
 '20230301',
 '20230228',
 '20230227',
 '20230224',
 '20230223',
 '20230222',
 '20230221',
 '20230220',
 '20230217',
 '20230216',
 '20230215',
 '20230214',
 '20230213',
 '20230210',
 '20230209',
 '20230208',
 '20230207',
 '20230206',
 '20230203',
 '20230202',
 '20230201',
 '20230131',
 '20230130',
 '20230120',
 '20230119',
 '20230118',
 '20230117',
 '20230116',
 '20230113',
 '20230112',
 '20230111',
 '20230110',
 '20230109',
 '20230106',
 '20230105',
 '20230104',
 '20230103',
 '20221230',
 '20221229',
 '20221228',
 '20221227',
 '20221226',
 '20221223',
 '20221222',
 '20221221',
 '20221220',
 '20221219',
 '20221216',
 '20221215',
 '20221214',
 '20221213',
 '20221212',
 '20221209',
 '20221208',
 '20221207',

3、循环日期，获取每日占比

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['小单主动买入占比', '中单主动买入占比', '大单主动买入占比'])

for date in date_list:
    sm_net_buy, md_net_buy, bg_net_buy = get_daily_buy_per(date)
    df.loc[date]=[sm_net_buy, md_net_buy, bg_net_buy]

df

In [ ]:
df.to_csv('主动买入成交占比.csv', encoding='utf-8')